# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import requests
import json
import csv
import datetime
import time
from citipy import citipy
import random
from scipy.stats import linregress

# Google API Key
from api_keys import g_key

## Generate Cities List

In [2]:
# Setup the ranges for latitude and longitude
latitude_range = (-90,90)
longatude_range = (-180,180)

# Create variables for data
cities = []

no_cities = 1500
for cntr in range(no_cities):
    latitude = random.uniform(latitude_range[0],latitude_range[1])
    longitude = random.uniform(longatude_range[0],longatude_range[1])
    
    # If we have chosen a latitude of 0 then we don't want to use that
    # one
    if latitude != 0:
        city = citipy.nearest_city(latitude,longitude)
        city_name = city.city_name
        city_country = city.country_code
        if not any(c['name'] == city_name and c['country'] == city_country for c in cities):
            city_dict = {'name': city_name,'country': city_country}
            cities.append(city_dict)

In [3]:
print(cities)

[{'name': 'kintampo', 'country': 'gh'}, {'name': 'clyde river', 'country': 'ca'}, {'name': 'zabol', 'country': 'ir'}, {'name': 'bluff', 'country': 'nz'}, {'name': 'saskylakh', 'country': 'ru'}, {'name': 'hermanus', 'country': 'za'}, {'name': 'bengkulu', 'country': 'id'}, {'name': 'mendeleyevsk', 'country': 'ru'}, {'name': 'ushuaia', 'country': 'ar'}, {'name': 'albany', 'country': 'au'}, {'name': 'illoqqortoormiut', 'country': 'gl'}, {'name': 'cabo san lucas', 'country': 'mx'}, {'name': 'penticton', 'country': 'ca'}, {'name': 'provideniya', 'country': 'ru'}, {'name': 'tuktoyaktuk', 'country': 'ca'}, {'name': 'samusu', 'country': 'ws'}, {'name': 'nelson bay', 'country': 'au'}, {'name': 'hilo', 'country': 'us'}, {'name': 'busselton', 'country': 'au'}, {'name': 'hurricane', 'country': 'us'}, {'name': 'samut songkhram', 'country': 'th'}, {'name': 'cairns', 'country': 'au'}, {'name': 'mataura', 'country': 'pf'}, {'name': 'port hardy', 'country': 'ca'}, {'name': 'cape town', 'country': 'za'},

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [4]:
base_url = "http://api.openweathermap.org/data/2.5/weather?"
units = "imperial"
city_id = []
city_name = []
city_cloudiness = []
city_country = []
city_date = []
city_humidity = []
city_latitude = []
city_longitude = []
city_temperature = []
city_wind_speed = []


# Build partial query URL
query_url = f"{base_url}appid={g_key}&units={units}&q="

city_cntr = -1
no_cities_to_retrieve = 525


# Loop through cities
for index, city in enumerate(cities):
    # See if we need to sleep so we don't go over our 60 a minute max for the api
    q, mod = divmod(index+1,55)
    if mod == 0:
        print("Pausing for a minute so we don't max out our api per minutes limit")
        time.sleep(60)
    
    
    
    # Retrieve data information
    try:
        city_json = requests.get(query_url + city['name'] + ', ' + city['country']).json()
    except:
        print(f"City not found - {city['name']}, {city['country']} - skipping")

    if city_json['cod'] == 200:
        city_cntr += 1
        print(f"Retrieved data for {city_cntr}-{city_json['name']}, {city_json['sys']['country']}({city_json['coord']['lat']},{city_json['coord']['lon']})")
        
        city_id.append(city_cntr)
        city_name.append(city_json['name'])
        city_cloudiness.append(city_json['clouds']['all'])
        city_country.append(city_json['sys']['country'])
        city_date.append(city_json['dt'])
        city_humidity.append(city_json['main']['humidity'])
        city_latitude.append(city_json['coord']['lat'])
        city_longitude.append(city_json['coord']['lon'])
        city_temperature.append(city_json['main']['temp_max'])              
        city_wind_speed.append(city_json['wind']['speed'])
    
    if city_cntr+1 == no_cities_to_retrieve:
        break
    
          

Retrieved data for 0-Kintampo,  GH(8.06,-1.73)
Retrieved data for 1-Clyde River,  CA(70.47,-68.59)
Retrieved data for 2-Zabol,  IR(31.03,61.5)
Retrieved data for 3-Bluff,  NZ(-46.6,168.33)
Retrieved data for 4-Saskylakh,  RU(71.92,114.08)
Retrieved data for 5-Hermanus,  ZA(-34.42,19.23)
Retrieved data for 6-Bengkulu,  ID(-3.8,102.27)
Retrieved data for 7-Mendeleyevsk,  RU(55.91,52.29)
Retrieved data for 8-Ushuaia,  AR(-54.8,-68.3)
Retrieved data for 9-Albany,  AU(-35.02,117.89)
Retrieved data for 10-Cabo San Lucas,  MX(22.89,-109.91)
Retrieved data for 11-Penticton,  CA(49.48,-119.59)
Retrieved data for 12-Provideniya,  RU(64.38,-173.3)
Retrieved data for 13-Tuktoyaktuk,  CA(69.45,-133.04)
Retrieved data for 14-Nelson Bay,  AU(-32.72,152.15)
Retrieved data for 15-Hilo,  US(19.73,-155.09)
Retrieved data for 16-Busselton,  AU(-33.65,115.33)
Retrieved data for 17-Hurricane,  US(37.18,-113.29)
Retrieved data for 18-Samut Songkhram,  TH(13.4,99.96)
Retrieved data for 19-Cairns,  AU(-16.92,1

Retrieved data for 161-Georgetown,  SH(-7.93,-14.42)
Retrieved data for 162-Cantagallo,  CO(7.38,-73.92)
Retrieved data for 163-Araguaçu,  BR(-12.93,-49.83)
Retrieved data for 164-Vao,  NC(-22.67,167.48)
Retrieved data for 165-Xixiang,  CN(35.16,112.86)
Retrieved data for 166-Paraiso,  MX(18.4,-93.23)
Retrieved data for 167-General Roca,  AR(-39.03,-67.58)
Retrieved data for 168-Touros,  BR(-5.2,-35.46)
Retrieved data for 169-Longyearbyen,  SJ(78.22,15.64)
Retrieved data for 170-Ukiah,  US(39.15,-123.21)
Retrieved data for 171-Thompson,  CA(55.74,-97.86)
Retrieved data for 172-Palmer,  US(42.16,-72.33)
Retrieved data for 173-Nikolskoye,  RU(59.7,30.79)
Retrieved data for 174-Arti,  RU(56.42,58.53)
Retrieved data for 175-Tahe,  CN(52.33,124.73)
Retrieved data for 176-Bafoulabé,  ML(13.81,-10.83)
Retrieved data for 177-Cockburn Town,  TC(21.46,-71.14)
Retrieved data for 178-Naze,  JP(28.37,129.48)
Retrieved data for 179-Taltal,  CL(-25.4,-70.48)
Retrieved data for 180-Sigli,  ID(5.38,95.

Retrieved data for 318-Allāpalli,  IN(19.42,80.07)
Retrieved data for 319-Haines Junction,  CA(60.75,-137.51)
Retrieved data for 320-Kruševac,  RS(43.58,21.33)
Retrieved data for 321-Baião,  BR(-2.79,-49.67)
Retrieved data for 322-Arrondissement de Limoges,  FR(45.75,1.33)
Retrieved data for 323-Porto Walter,  BR(-8.27,-72.74)
Retrieved data for 324-Ancud,  CL(-41.87,-73.82)
Retrieved data for 325-Mae Hong Son,  TH(19.3,97.97)
Retrieved data for 326-Henties Bay,  NA(-22.12,14.28)
Retrieved data for 327-Eureka,  US(40.8,-124.16)
Retrieved data for 328-Kaitangata,  NZ(-46.28,169.85)
Retrieved data for 329-Shitanjing,  CN(39.23,106.34)
Retrieved data for 330-Grootfontein,  NA(-19.57,18.12)
Retrieved data for 331-Baykit,  RU(61.67,96.37)
Retrieved data for 332-Shenjiamen,  CN(29.96,122.3)
Retrieved data for 333-Grand Forks,  CA(49.03,-118.44)
Retrieved data for 334-Tiksi,  RU(71.69,128.87)
Retrieved data for 335-Cartagena,  PH(9.82,122.4)
Retrieved data for 336-Qasigiannguit,  GL(68.82,-51

Retrieved data for 478-Banyo,  CM(6.75,11.82)
Retrieved data for 479-Lompoc,  US(34.64,-120.46)
Retrieved data for 480-Camalú,  MX(30.85,-116.07)
Retrieved data for 481-Baiyin,  CN(36.56,104.21)
Retrieved data for 482-Havre-St-Pierre,  CA(50.23,-63.6)
Retrieved data for 483-Takoradze,  GH(4.88,-1.76)
Retrieved data for 484-Gorin,  RU(51.2,136.67)
Retrieved data for 485-Kitui,  KE(-1.37,38.01)
Retrieved data for 486-Terney,  RU(45.05,136.61)
Retrieved data for 487-Katsuura,  JP(35.13,140.3)
Retrieved data for 488-Luzhou,  CN(28.87,105.44)
Retrieved data for 489-Kourou,  GF(5.16,-52.65)
Pausing for a minute so we don't max out our api per minutes limit
Retrieved data for 490-Chapais,  CA(49.78,-74.85)
Retrieved data for 491-Port Blair,  IN(11.67,92.75)
Retrieved data for 492-Ust'-Karsk,  RU(52.7,118.81)
Retrieved data for 493-Aklavik,  CA(68.22,-135.01)
Retrieved data for 494-Río Gallegos,  AR(-51.62,-69.22)
Retrieved data for 495-Vardø,  NO(70.37,31.11)
Retrieved data for 496-Tevaitoa, 

### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

## Latitude vs. Humidity Plot

## Latitude vs. Cloudiness Plot

## Latitude vs. Wind Speed Plot

## Linear Regression

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression